## Call the cython callback from another module using raw address and additional arguments

In [1]:
%load_ext cython

In [2]:
%%cython
# The first (core) module - caller

ctypedef int (*TFunc)(int x, dict kwargs)

def prun(long callback_addr, int x, **kwargs):
    """Python wrapper to call the func by raw address"""
    cdef TFunc func = <TFunc>callback_addr
    return func(x, kwargs)

In [3]:
%%cython
# The second (user) module - callee

cdef int callback(int x, dict kwargs):
    """Internal cython format function (that can use stdlib parameters directly)"""
    data = kwargs['result']
    data[0,0] = x
    return x+x

def callback_addr():
    """Exports raw address of callback to python"""
    return <long>&callback

In [4]:
callback_addr()

140046856955696

In [5]:
import numpy as np

X = np.zeros((10, 10))
X

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
prun(callback_addr(), 4, result=X)
X

array([[4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
%timeit prun(callback_addr(), 4, result=X)

718 ns ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
